### Import

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

### Data Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/open/train.csv').drop(columns=['ID'])
test = pd.read_csv('/content/drive/MyDrive/open/test.csv').drop(columns=['ID'])

In [ ]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [ ]:
analysis_results = {}
for col in categorical_columns:
    if col in train.columns:
        value_counts = train[col].value_counts(dropna=False)  # NaN 포함하여 개수 계산
        missing_values = train[col].isna().sum()  # 결측값 개수

        analysis_results[col] = {
            "Value Counts": value_counts.to_dict(),
            "Missing Values": missing_values
        }

In [ ]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [ ]:
for col, result in analysis_results.items():
    print(f"컬럼: {col}")
    print("값 개수:", result["Value Counts"])
    print("결측값 개수:", result["Missing Values"])
    print("-" * 50)

컬럼: 시술 시기 코드
값 개수: {'TRDQAZ': 38969, 'TRCMWS': 38090, 'TRYBLT': 36713, 'TRVNRY': 36173, 'TRJXFG': 36031, 'TRZKPL': 35544, 'TRXQMD': 34831}
결측값 개수: 0
--------------------------------------------------
컬럼: 시술 당시 나이
값 개수: {'만18-34세': 102476, '만35-37세': 57780, '만38-39세': 39247, '만40-42세': 37348, '만43-44세': 12253, '만45-50세': 6918, '알 수 없음': 329}
결측값 개수: 0
--------------------------------------------------
컬럼: 임신 시도 또는 마지막 임신 경과 연수
값 개수: {nan: 246981, 7.0: 1214, 8.0: 1176, 9.0: 1023, 6.0: 991, 10.0: 947, 11.0: 701, 5.0: 692, 12.0: 510, 13.0: 481, 14.0: 331, 4.0: 305, 15.0: 254, 16.0: 202, 17.0: 140, 18.0: 106, 3.0: 89, 19.0: 86, 20.0: 83, 2.0: 21, 0.0: 12, 1.0: 6}
결측값 개수: 246981
--------------------------------------------------
컬럼: 시술 유형
값 개수: {'IVF': 250060, 'DI': 6291}
결측값 개수: 0
--------------------------------------------------
컬럼: 특정 시술 유형
값 개수: {'ICSI': 122368, 'IVF': 91755, 'Unknown': 26939, 'IUI': 6100, 'ICSI:ICSI': 2314, 'ICSI / BLASTOCYST ': 1609, 'IVF / BLASTOCYST': 1248, 'IVF:IVF

In [ ]:
df = train

# PGD 시술 여부 및 PGS 시술 여부별 임신 성공률 집계
pgd_pgs_analysis = df.groupby(["PGD 시술 여부", "PGS 시술 여부"])["임신 성공 여부"].value_counts(normalize=True).unstack()

# 결과 출력
print(pgd_pgs_analysis)

# CSV로 저장 (선택 사항)
pgd_pgs_analysis.to_csv("pgd_pgs_pregnancy_analysis.csv", encoding="utf-8-sig")

Empty DataFrame
Columns: []
Index: []


In [ ]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [ ]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [ ]:
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

### Train

In [ ]:
model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

ExtraTreesClassifier(random_state=42)

### Predict

In [ ]:
pred_proba = model.predict_proba(test_encoded)[:, 1]

### Submission

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/open/sample_submission.csv')
sample_submission['probability'] = pred_proba

In [ ]:
sample_submission.to_csv('./baseline_submit_1.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# 데이터 로드 및 ID 컬럼 제거
train = pd.read_csv('/content/drive/MyDrive/open/train.csv').drop(columns=['ID'])
test = pd.read_csv('/content/drive/MyDrive/open/test.csv').drop(columns=['ID'])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
모델 학습 완료!


결측값 70%이상 되는 피쳐 지움
Ordinal Encoder
ExtraTree Classifier

In [ ]:
print(train.columns)

Index(['시술 시기 코드', '시술 당시 나이', '임신 시도 또는 마지막 임신 경과 연수', '시술 유형', '특정 시술 유형',
       '배란 자극 여부', '배란 유도 유형', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부',
       '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인',
       '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환',
       '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제',
       '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인',
       '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아 생성 주요 이유', '총 시술 횟수',
       '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수',
       'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수',
       '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수',
       '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수',
       '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 출처', '정자 출처',
       '난자 기증자 나이', '정자 기증자 나이', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부',
       '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자

In [ ]:
train.columns = train.columns.str.strip()
test.columns = test.columns.str.strip()

In [ ]:
print(f"실제 '임신 성공 여부' 컬럼 존재 여부: {'임신 성공 여부' in train.columns}")
print("컬럼 목록:", list(train.columns))

실제 '임신 성공 여부' 컬럼 존재 여부: True
컬럼 목록: ['시술 시기 코드', '시술 당시 나이', '임신 시도 또는 마지막 임신 경과 연수', '시술 유형', '특정 시술 유형', '배란 자극 여부', '배란 유도 유형', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아 생성 주요 이유', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 채취 경과일', '난자 해동 경과일', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일',

In [ ]:
train.rename(columns={'임신 성공 여부 ': '임신 성공 여부'}, inplace=True)
test.rename(columns={'임신 성공 여부 ': '임신 성공 여부'}, inplace=True)

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import f1_score

# 학습 데이터셋 설정
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

# 제외할 컬럼 리스트
columns_to_remove = [
    "PGD 시술 여부", "PGS 시술 여부",
    "난자 채취 경과일", "난자 해동 경과일",
    "난자 혼합 경과일", "배아 해동 경과일"
]

# 해당 컬럼 제거
X = X.drop(columns=columns_to_remove, errors='ignore')
test = test.drop(columns=columns_to_remove, errors='ignore')

# 범주형 변수 리스트 (제거된 변수 제외)
# categorical_columns = [
#     "시술 시기 코드", "시술 당시 나이", "시술 유형", "특정 시술 유형",
#     "배란 자극 여부", "배란 유도 유형", "단일 배아 이식 여부",
#     "착상 전 유전 진단 사용 여부", "남성 주 불임 원인", "남성 부 불임 원인",
#     "여성 주 불임 원인", "여성 부 불임 원인", "부부 주 불임 원인",
#     "부부 부 불임 원인", "불명확 불임 원인", "불임 원인 - 난관 질환",
#     "불임 원인 - 남성 요인", "불임 원인 - 배란 장애", "불임 원인 - 여성 요인",
#     "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증", "불임 원인 - 정자 농도",
#     "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태",
#     "배아 생성 주요 이유", "총 시술 횟수", "클리닉 내 총 시술 횟수",
#     "IVF 시술 횟수", "DI 시술 횟수", "총 임신 횟수", "IVF 임신 횟수",
#     "DI 임신 횟수", "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수",
#     "난자 출처", "정자 출처", "난자 기증자 나이", "정자 기증자 나이",
#     "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부"
# ]
categorical_columns = [
    "시술 시기 코드", "시술 당시 나이", "특정 시술 유형",
    "배란 자극 여부", "배란 유도 유형", "단일 배아 이식 여부",
    "착상 전 유전 진단 사용 여부", "남성 주 불임 원인", "남성 부 불임 원인",
    "여성 주 불임 원인", "여성 부 불임 원인", "부부 주 불임 원인",
    "부부 부 불임 원인", "불명확 불임 원인", "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
     "불임 원인 - 자궁내막증", "불임 원인 - 정자 농도",
    "배아 생성 주요 이유", "총 시술 횟수", "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수", "DI 시술 횟수", "총 임신 횟수", "IVF 임신 횟수",
    "DI 임신 횟수", "총 출산 횟수", "IVF 출산 횟수",
    "난자 출처", "정자 출처", "난자 기증자 나이", "정자 기증자 나이",
    "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부"
]
# 실제 존재하는 컬럼만 추출
categorical_columns = [col for col in categorical_columns if col in X.columns]

# 카테고리형 변수를 문자열로 변환 (Ordinal Encoding을 위해)
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

# Ordinal Encoding 적용
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
X_encoded = X.copy()
test_encoded = test.copy()

X_encoded[categorical_columns] = encoder.fit_transform(X[categorical_columns])
test_encoded[categorical_columns] = encoder.transform(test[categorical_columns])

# 결측값 채우기 (숫자형 변수)
numeric_columns = X.select_dtypes(include=[np.number]).columns.tolist()
X_encoded[numeric_columns] = X_encoded[numeric_columns].fillna(0)
test_encoded[numeric_columns] = test_encoded[numeric_columns].fillna(0)

# Voting Classifier를 위한 개별 모델 정의
models = [
    ('random_forest', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('adaboost', AdaBoostClassifier(n_estimators=100, random_state=42)),
    ('gradient_boosting', GradientBoostingClassifier(n_estimators=100, random_state=42)),
    ('logistic_regression', LogisticRegression(max_iter=500))
]

# Voting Classifier 정의 (soft Voting)
voting_clf_new = VotingClassifier(estimators=models, voting='soft')

voting_clf_new.fit(X_encoded, y)


In [ ]:


#  확률 예측 수행 (Soft Voting 모델 사용)
pred_proba = voting_clf_new.predict_proba(test_encoded)[:, 1]

#  제출 파일 로드
sample_submission = pd.read_csv('/content/drive/MyDrive/open/sample_submission.csv')

#  예측 확률 저장
sample_submission['probability'] = pred_proba

# CSV로 저장
sample_submission.to_csv('./voting_soft_submit.csv', index=False)

print("Soft Voting 모델의 예측 확률 저장 완료")

✅ Soft Voting 모델의 예측 확률 저장 완료! (submission 파일 생성됨)
